In [4]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI

/Users/soroushav/Desktop/projects/LLM_Engineering/expert-knowledge-worker/llm_basics/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [45]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = api_key
openai = OpenAI()

In [32]:
MODEL = "gpt-4o-mini"

In [8]:
context = {}
employees = glob.glob("knowledge-base/employees/*")
for employee in employees:
    name = employee.split(" ")[-1][:-3]
    doc = ""
    with open(employee, "r") as f:
        doc = f.read()
    context[name] = doc

In [17]:
products = glob.glob("knowledge-base/products/*")
for product in products:
    name = product.split("/")[-1][:-3]
    doc = ""
    with open(product, "r") as f:
        doc = f.read()
    context[name] = doc


In [18]:
system_message = "You are an expert in answering accurate questions about Insurellm, the Insurance Tech company. Give brief, accurate answers. If you don't know the answer, say so. Do not make anything up if you haven't been provided with relevant context."

In [37]:
def get_relevant_context(message):
    relevant_context = []
    for context_keys, context_details in context.items():
        if context_keys in message:
            relevant_context.append(context_details)
    return relevant_context

In [25]:
get_relevant_context("Lancaster")

["# Avery Lancaster\n\n## Summary\n- **Date of Birth**: March 15, 1985  \n- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  \n- **Location**: San Francisco, California  \n\n## Insurellm Career Progression\n- **2015 - Present**: Co-Founder & CEO  \n  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  \n\n- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  \n  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  \n\n- **2010 - 2013**: Business Analyst at Edge Analytics  \n  Prior to joining Innovate, Avery worked as a Business Analyst, focusing on market trends and consumer preferen

In [38]:
def add_context(message):
    relevant_context = get_relevant_context(message)
    print(relevant_context)
    if relevant_context:
        message += "\n\nThe following additional context might be relevant in answering this question:\n\n"
        for relevant in relevant_context:
            message += f"\n\n{relevant}"
    return message

In [ ]:
add_context("who is Avery Lancaster?")

["# Avery Lancaster\n\n## Summary\n- **Date of Birth**: March 15, 1985  \n- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  \n- **Location**: San Francisco, California  \n\n## Insurellm Career Progression\n- **2015 - Present**: Co-Founder & CEO  \n  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  \n\n- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  \n  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  \n\n- **2010 - 2013**: Business Analyst at Edge Analytics  \n  Prior to joining Innovate, Avery worked as a Business Analyst, focusing on market trends and consumer preferen

"who is Avery Lancaster?\n\nThe following additional context might be relevant in answering this question:\n\n\n\n# Avery Lancaster\n\n## Summary\n- **Date of Birth**: March 15, 1985  \n- **Job Title**: Co-Founder & Chief Executive Officer (CEO)  \n- **Location**: San Francisco, California  \n\n## Insurellm Career Progression\n- **2015 - Present**: Co-Founder & CEO  \n  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  \n\n- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  \n  Before launching Insurellm, Avery was a leading Senior Product Manager at Innovate Insurance Solutions, where she developed groundbreaking insurance products aimed at the tech sector.  \n\n- **2010 - 2013**: Business Analyst at Edge Analytics  \n

In [47]:
def chat(message, history):
    messages = [{"role": "system", "message": system_message}] + history
    print("messages areeeeeeeë:\n")
    print(messages)
    user_message = add_context(message)
    messages.append({"role": "user", "message": user_message})

    stream = openai.chat.completions.create(messages=messages, model=MODEL, stream=True)
    reply = ""
    for chunk in stream:
        reply += chunk.choices[0].delta.context or ""
        yield reply

In [48]:
view = gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


messages areeeeeeeë:

[{'role': 'system', 'message': "You are an expert in answering accurate questions about Insurellm, the Insurance Tech company. Give brief, accurate answers. If you don't know the answer, say so. Do not make anything up if you haven't been provided with relevant context."}]
[]


Traceback (most recent call last):
  File "/Users/soroushav/Desktop/projects/LLM_Engineering/expert-knowledge-worker/llm_basics/venv/lib/python3.12/site-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/soroushav/Desktop/projects/LLM_Engineering/expert-knowledge-worker/llm_basics/venv/lib/python3.12/site-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/soroushav/Desktop/projects/LLM_Engineering/expert-knowledge-worker/llm_basics/venv/lib/python3.12/site-packages/gradio/blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/soroushav/Desktop/projects/LLM_Engineering/expert-knowledge-worker/llm_basics/venv/lib/python3.12/site-packages/gradio/blocks.py", line 1635, i